
Project Facemask Detection 


In [4]:
import torch


#Checking for availablilty of CUDA
train_on_gpu = torch.cuda.is_available()
use_cuda = train_on_gpu

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
import os
import numpy as np
import torchvision
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import splitfolders


datapath = 'D:\TarunDocs\Kaggle\data'


splitfolders.ratio(datapath, output="D:\TarunDocs\Kaggle\data\splited", seed=1337, ratio=(.8, .1, .1), group_prefix=None)


Newpath = 'D:/TarunDocs/Kaggle/data/splited'

trainpath = os.path.join(Newpath,'train/')
valpath = os.path.join(Newpath,'val/')
testpath = os.path.join(Newpath,'test/')

Copying files: 7553 files [00:11, 633.86 files/s]


In [6]:
%matplotlib inline

transform_train = transforms.Compose([transforms.Resize((224,224)),transforms.RandomHorizontalFlip(p=0.5),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),transforms.ToTensor()])

transform_val = transforms.Compose([transforms.Resize((224,224)),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),transforms.ToTensor()])

In [9]:
train_data = datasets.ImageFolder(trainpath, transform=transform_train)
val_data = datasets.ImageFolder(valpath,transform=transform_val)
test_data = datasets.ImageFolder(testpath,transform=transform_val)

In [15]:
# define dataloader parameters
batch_size = 20
num_workers=0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

In [38]:
loaders = {'train': train_loader, 'valid': val_loader, 'test': test_loader}

In [20]:
vgg16 = models.vgg16(pretrained = True)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\admin/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [01:14<00:00, 7.41MB/s]


In [21]:
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [31]:
print(vgg16.classifier[6])

Linear(in_features=4096, out_features=2, bias=True)


In [32]:

# Freeze training for all "features" layers
for param in vgg16.features.parameters():
    param.requires_grad = False

In [33]:
# creating unique FC for the output
import torch.nn as nn
last_layer = nn.Linear(vgg16.classifier[6].in_features,2)

vgg16.classifier[6]=last_layer
if train_on_gpu:
    vgg16.cuda()
    pass
print(vgg16.classifier[6].out_features)

2


Training

In [34]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
def get_optimizer_transfer(model):
    optimizer = torch.optim.SGD(model.classifier.parameters(), lr = 0.001)
    return optimizer

In [35]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            
            

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            output = model(data)
            loss = criterion(output, target)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))

    
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        ## TODO: if the validation loss has decreased, save the model at the filepath stored in save_path
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

        
        
    return model

In [ ]:
vgg16 = train(20, loaders, vgg16, get_optimizer_transfer(vgg16),criterion, use_cuda, 'face_detectvgg16.pt')
vgg16.load_state_dict(torch.load('face_detectvgg16.pt'))

In [36]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)